In [1]:
!pip install pandas
!pip install pandasql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import pandasql as ps

csv_name = 'cars.csv'

cars = pd.DataFrame(
    {
        'speed': [300, 200, 140],
        'engine': [5, 0, 3],
        'doors': [2, 4, 5],
    }, index=pd.Index(['lambo', 'tesla', 'lr'], name='car')
)

cars

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


In [3]:
cars.to_csv(csv_name)

In [4]:
cars = pd.read_csv(csv_name, index_col='car')
cars_wo_index = pd.read_csv(csv_name)

In [5]:
cars
cars_wo_index

,speed,engine,doors
car,,,
lambo,300,5,2
tesla,200,0,4
lr,140,3,5


,car,speed,engine,doors
0,lambo,300,5,2
1,tesla,200,0,4
2,lr,140,3,5


In [6]:
ti = pd.read_csv('titanic.csv', index_col=0)
ti

,Name,PClass,Age,Sex,Survived,SexCode
PassengerID,,,,,,
1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0
...,...,...,...,...,...,...
1309,"Zakarian, Mr Artun",3rd,27.00,male,0,0
1310,"Zakarian, Mr Maprieder",3rd,26.00,male,0,0
1311,"Zenni, Mr Philip",3rd,22.00,male,0,0


In [7]:
sql = """
select 
       --sex, survived,
       count(Name), count(Age), count(SexCode), count(*)
  from ti
--  group by sex, survived
--  order by sex, survived
"""
sql1 = """
select case when age is null then 'null' else 'not null' end as age_is_null, count(*)
  from ti
  group by case when age is null then 'null' else 'not null' end
"""

In [8]:
ps.sqldf(sql)
ps.sqldf(sql1)
ps.sqldf("select * from ti where age is null limit 5")
ps.sqldf("select * from ti where age is not null limit 5")

,count(Name),count(Age),count(SexCode),count(*)
0,1313,756,1313,1313


,age_is_null,count(*)
0,not null,756
1,null,557


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,13,"Aubert, Mrs Leontine Pauline",1st,None,female,1,1
1,14,"Barkworth, Mr Algernon H",1st,None,male,1,0
2,15,"Baumann, Mr John D",1st,None,male,0,0
3,30,"Borebank, Mr John James",1st,None,male,0,0
4,33,"Bradley, Mr George",1st,None,male,1,0


,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
1,2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
2,3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
3,4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
4,5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0


In [9]:
ps.sqldf("select * from ti where PClass='*'")

,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,457,Jacobsohn Mr Samuel,*,None,male,0,0


In [10]:
sql_surv_class = """
select 
       PClass, survived, count(*)
  from ti
  group by PClass, survived
  order by 1, 2
"""

In [11]:
ps.sqldf(sql_surv_class)

,PClass,Survived,count(*)
0,*,0,1
1,1st,0,129
2,1st,1,193
3,2nd,0,160
4,2nd,1,119
5,3rd,0,573
6,3rd,1,138


In [12]:
ti.pivot_table(
    index=['Sex'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Sex,,,
female,38.0,28.0,22.0
male,42.0,28.0,25.5


In [13]:
ti.pivot_table(
    index=['Survived'],
    columns=['PClass'],
    values='Age',
    aggfunc='median',
)

PClass,1st,2nd,3rd
Survived,,,
0,46.0,29.5,25.0
1,36.0,25.0,22.0
